In [3]:
import findspark
findspark.init()

In [4]:
from pyspark import SparkContext, SparkConf

In [5]:
conf=SparkConf().setAppName("BDProject").setMaster("local[3]")

In [6]:
sc=SparkContext(conf=conf)

In [16]:
rawTags=sc.textFile("travel.stackexchange.com\Badges.xml")

In [17]:
rawTags.take(10)

['  <row Id="1" UserId="2" Name="Autobiographer" Date="2011-06-21T20:16:48.910" Class="3" TagBased="False" />',
 '  <row Id="2" UserId="3" Name="Autobiographer" Date="2011-06-21T20:16:48.910" Class="3" TagBased="False" />',
 '  <row Id="3" UserId="4" Name="Autobiographer" Date="2011-06-21T20:16:48.910" Class="3" TagBased="False" />',
 '  <row Id="4" UserId="6" Name="Autobiographer" Date="2011-06-21T20:21:49.157" Class="3" TagBased="False" />',
 '  <row Id="5" UserId="8" Name="Autobiographer" Date="2011-06-21T20:21:49.157" Class="3" TagBased="False" />',
 '  <row Id="6" UserId="9" Name="Autobiographer" Date="2011-06-21T20:21:49.157" Class="3" TagBased="False" />',
 '  <row Id="7" UserId="10" Name="Autobiographer" Date="2011-06-21T20:21:49.173" Class="3" TagBased="False" />',
 '  <row Id="8" UserId="12" Name="Autobiographer" Date="2011-06-21T20:21:49.173" Class="3" TagBased="False" />',
 '  <row Id="9" UserId="13" Name="Autobiographer" Date="2011-06-21T20:21:49.173" Class="3" TagBased="F

In [135]:
def cleaning(listIn):
    listOut=[]
    for x in listIn:
        if listIn.index(x)%2==0:
            listOut.append(x.replace(' ','').replace('<','').replace('=',''))
        else:
            listOut.append(x)
    return {listOut[i]:listOut[i+1] for i in range(0,len(listOut)-1,2)}

In [133]:
tags=rawTags.filter(lambda x: x.startswith('  <row Id=')).\
map(lambda x: x.split('"')).map(cleaning)

In [134]:
tags.take(10)

[{'Class': '3',
  'Date': '2011-06-21T20:16:48.910',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '2',
  'rowId': '1'},
 {'Class': '3',
  'Date': '2011-06-21T20:16:48.910',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '3',
  'rowId': '2'},
 {'Class': '3',
  'Date': '2011-06-21T20:16:48.910',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '4',
  'rowId': '3'},
 {'Class': '3',
  'Date': '2011-06-21T20:21:49.157',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '6',
  'rowId': '4'},
 {'Class': '3',
  'Date': '2011-06-21T20:21:49.157',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '8',
  'rowId': '5'},
 {'Class': '3',
  'Date': '2011-06-21T20:21:49.157',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '9',
  'rowId': '6'},
 {'Class': '3',
  'Date': '2011-06-21T20:21:49.173',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '10',
  'rowId': '7'},
 {'Class': '3',
  'Date': '2011-0